In [ ]:
# -*- coding: utf-8 -*-
import csv, json, time, urllib.parse, urllib.request
from urllib.error import HTTPError
GEO_THROTTLE=0.1; OSRM_THROTTLE=0.2
GEOCODE_CACHE={}; _last=0.0
def throttle(sec):
    global _last
    dt=time.time()-_last
    if dt<sec: time.sleep(sec-dt)
    _last=time.time()
def fetch_json(url, timeout=30):
    try:
        with urllib.request.urlopen(url, timeout=timeout) as r:
            return json.load(r)
    except Exception as e:
        print(f"[warn] {e}: {url}")
        return None
def geocode_gouv(addr):
    if addr in GEOCODE_CACHE: return GEOCODE_CACHE[addr]
    throttle(GEO_THROTTLE)
    q=urllib.parse.urlencode({'q':addr,'limit':1})
    url=f"https://api-adresse.data.gouv.fr/search/?{q}"
    data=fetch_json(url)
    if data and data.get('features'):
        lon,lat=data['features'][0]['geometry']['coordinates']
        GEOCODE_CACHE[addr]=(lat,lon)
        return lat,lon
    GEOCODE_CACHE[addr]=(None,None)
    return None,None
def route_distance_osrm(a_lat,a_lon,b_lat,b_lon):
    throttle(OSRM_THROTTLE)
    coords=f"{a_lon},{a_lat};{b_lon},{b_lat}"
    url=f"https://router.project-osrm.org/route/v1/driving/{coords}?overview=false"
    data=fetch_json(url)
    if data and data.get('routes'):
        return round(data['routes'][0]['distance']/1000,2)
    return None


In [ ]:
from google.colab import files
uploaded = files.upload()
csv_file = next(iter(uploaded.keys()))
print('Fichier reçu :', csv_file)


In [ ]:
rows = list(csv.DictReader(open(csv_file, encoding='utf-8-sig')))
out = csv_file.replace('.csv','_with_coords.csv')
w = csv.DictWriter(open(out,'w',encoding='utf-8',newline=''),
                    fieldnames=list(rows[0].keys())+['Lat départ','Lon départ','Lat destination','Lon destination','Distance_km'])
w.writeheader()
total=len(rows)
for i,row in enumerate(rows,1):
    print(f'Processing {i}/{total} ID={row.get("Id","?")}')
    lat1,lon1=geocode_gouv(row['Adresse départ'])
    lat2,lon2=geocode_gouv(row['Adresse destination'])
    row['Lat départ'],row['Lon départ']=lat1 or '',lon1 or ''
    row['Lat destination'],row['Lon destination']=lat2 or '',lon2 or ''
    if lat1 and lon1 and lat2 and lon2:
        row['Distance_km']=route_distance_osrm(lat1,lon1,lat2,lon2) or ''
    else:
        row['Distance_km']=''
    w.writerow(row)
print('Done :', out)


In [ ]:
files.download(out)
